<a href="https://colab.research.google.com/github/ynaowusu/protein-folding-quantum-algorithms/blob/main/proteinfolding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit #this is just me intalling qiskit into our notebook
!pip install matplotlib plotly #since it says we need a 3d structure to simulate the lattice and any other 3d elements
!pip install numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 9.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random


In [3]:
"""
        proteins fold in 3D space and here we're working with a simplified lattice model w tetrahedral directions. There are two kinds of alternating sites in the protein chain: 'A' and 'B'.
        every turn corresponds to a direction in 3D space either x,y or z
        The vectors  [1, 1, 1]) are unit steps in different directions based on the protein structure model.
"""

class ProtienFoldVector:
  def __init_(self):
    self.lattice_vector = {
        "A":{
            0:[(1,1,1)], #|00> This is for the first round of qubits reprsents x,y,z
            1:[(1,-1,-1)], #|10>
            2:[(-1,1,-1)], #|01>
            3:[(-1,-1,-1)] #|11>
        },
        "B":{
            0:[(1,1,-1)], #|00>
            1:[(1,-1,1)], #|10>
            2:[(-1,1,1)], #|01>
            3:[(-1,-1,-1)] #11>
        }





    }

    pass


    #vectors go here


In [4]:
#hamiltonian functions


# t_i and t_j are vectors
# we decode the bit string and supply indices i and j directly.
#for each pair of beads i and j, T(i,j ) returns a 1 iff the turns t1 and tj are along the same axis
#we need this for the growth constrainst function
#we need this to prevrent the growth of the chain towards unphysical geometries



def T(t_i,t_j):
  if np.array_equal(t_i,t_j) or np.array_equal(t_i, -t_j) :
    return 1
  else:
    return 0

def test_T_function():
  assert T(np.array([1,1,1]),np.array([1,1,1]))== 1
  assert T(np.array([-1,-1,-1]),np.array([-1,-1,-1]))== 1
  assert T(np.array([1,1,1]),np.array([-1,-1,-1]))== 1

  assert T(np.array([1,-1,-1]),np.array([1,1,-1]))== 0
  assert T(np.array([-1,1,-1]),np.array([-1,-1,-1]))== 0

  print("all test_t_function tests passed")
test_T_function()

'''
because there are somethings we do not have implemented yet, growth_constraint_hamiltonian is a bit hardcoded
what growth constaint does is it eliminates sequences where the same axis
is chosen twice in a row, since this will give rise to a chain folding back onto itself.
turns is for now a placeholder but it  supposed to be a chain and so we will decode the measured qubits to then get the backbone turns
for the particular fold and then if some consecutive pairs share an axis, then the penalty which here is 30 will apply
N is the length of the peptide chain which is the number of amino-acid residues

'''
amino_acid_sequence = ([1,2,3,4,5,6])
N = random.choice(amino_acid_sequence)
def growth_constraint_Hamiltonian():
  H_gc = 0
  penalty_weight = 30
  turns = [np.array([1,1,1]),np.array([1,1,1]), np.array([1,-1,-1]),np.array([-1,-1,1]),np.array([1,1,-1]),np.array([1, -1,-1]),np.array([-1,-1,1])]
  for i in range(3, N - 1):
     same_axis = T(turns[i], turns[i + 1])
     H_gc += 30 * same_axis
  return H_gc
growth_constraint_Hamiltonian()

all test_t_function tests passed


30

In [5]:
#the methods we creae will be used to fold protein model with 6 and 8
#amino acid sequences on 3D lattice
amino_acid_sequence = ([6,7,8])
N = random.choice(amino_acid_sequence)
print(N)

7


In [7]:

class CVARVQE:

    def __init__(self, hamiltonian, alpha = 0.1):
      #self is just the initializer
      #the hamiltonain in this case calcualtes the amount of energy it takes to formulate any given protien configuration.
      #this is important because the optimizer needs a way to evaluate how "good" or "bad" a quantum circuit’s result is given we need the lowest energy fold.
      #alpha = 0.1 is just instead of using all sampled energies, CVaR uses only the best-performing ones so bascially only sample the best 10% outputs/
      self.hamiltonian = hamiltonian #this is just so that we can call on the hamiltonian later on
      self.alpha = alpha #this just tells the computer only filter out the best 10 precent folds
      self.n_qubits = hamiltonian.total_qubits #what this does is it pull that number and save it in self.n_qubits, so we know how many qubits to use when building circuits later.
    def create_ansatz(self, params):
        qc = QuantumCircuit(self.n_qubits) #so this stores how many qubits are used that were passed in the self.n_qubits from the hamiltonian.
        #so the first step is to put the qubit into superpostion
        for i in range(self.n_qubits):
          qc.h(i) #the h is a hadamard gate and hate hadamard's do is put chaning each qubit into superpostion form. this loop is bascially saying loop through each qubit and add a H gate

          #then we need to add a rotation gate to each qubit. why do we need to do this? because this is how we teach the quanum cirucit how to fold a protien using the lowest energy possible.
          params_1 = 0 #We're looping through each qubit and applying a rotation gate to it. the rotation angle comes from a list called. we can keep track of where we are in the list using param_1 = 0
          # each number in this varible tells a qubit how much to rotate. with the params let you see how the exactly the protein is folding.
          for i in range(self.n_qubit):
            qc.rq(params[param_1], i) #this add the ry roatation to each qubit the params[params_1] gets the angle from the list of params_1
            #ry rotates the qubit on a Y-axis by a certain angl, these rotations are parameterized, which means that they can be changed and optimized.
            param_1 += 1 #after u use a parameter. move to the next one in the list



